### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from termcolor import colored
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Internal Hub _ Successful Connection!


### READ AND LIST FOLDER

In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Dong Nai Province
/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ho Chi Minh City
/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Long An Province
/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Binh Duong Province
/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ha Noi City


In [5]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Dong Nai Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ho Chi Minh City/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Long An Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Binh Duong Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ha Noi City/2022


In [6]:
#lấy quý
list_folder_sub2=[]
for i in list_folder_sub1:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub2=list_folder_sub2+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Dong Nai Province/2022/Q2
/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Dong Nai Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ho Chi Minh City/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Long An Province/2022/Q2
/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Long An Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Binh Duong Province/2022/Q2
/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Binh Duong Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ha Noi City/2022/Q3


In [9]:
#Provide information
Province_name=['Ha Noi City']#Chọn tỉnh cần import-- là tên folder 
Year=['2022']#Chọn năm cần import
Quarter=['Q3']#Chọn quý cần import
#Có thể chọn nhiều tỉnh, tên tỉnh để trong dấu nháy đơn và cách nhau bởi dấy phẩy. Áp dụng tương tự cho năm, quý
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub2:
    if i.split('/')[7] in Province_name and i.split('/')[8] in Year and i.split('/')[9] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ha Noi City/2022/Q3/PM_Apt_HN_20220930.xlsx


### GET IMPORT FILE

In [11]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[1].upper()
    if sector in ('APT'):#Chọn sector cần import: RETAIL, OFFICE, SA, HOTEL, APT, VLTH
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/Product mix/Flat file/Ha Noi City/2022/Q3/PM_Apt_HN_20220930.xlsx']

### IMPORT DATA

In [16]:
'''Prepare ingredients''' 
columns_that_need_unidecode=['Project_Name','Sub_Project_Name', 'Mix_Name'
                            ,'Sub_Project_Type', 'Project_City_Name', 'Project_District_Name']
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
#Create multi empty df for tracking autdit step
name_sector = ['APT', 'VLTH']
name_sector = [x.lower() for x in name_sector]
for i in name_sector:
    globals()['df_temp_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_flat_{}'.format(i)] = pd.DataFrame([])
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):  
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file   
    '''Check duplicate mix_name'''
    #data, df_dup = check_duplicate(data, 'Mix_Name')   
    #-------------------------------------------------------
    '''Validation step'''
    #Remove unfortmated values
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Type', 'Grade']
    lst_cls = ['Project_City_Name', 'Project_District_Name', 'Sub_Project_Type', 'Grade']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        #-------------------------------------------------------
        '''Import data process'''
        #Check project key
        data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
        processed_data, flag_key = check_project_key(file_url, data, sector, engine)
        if flag_key == 0:
            #Create tracking audit
            if sector == 'APT' or sector=='APARTMENT':
                df_temp_flat_apt = pd.concat([df_temp_flat_apt, data], axis=0)
                df_flat_apt = tracking_flat_file(df_temp_flat_apt, file_url)
            elif sector == 'VLTH':
                df_temp_flat_vlth = pd.concat([df_temp_flat_vlth, data], axis=0)
                df_flat_vlth = tracking_flat_file(df_temp_flat_vlth, file_url)
            else:
                pass
            #Get key 
            #data = get_project_key(flag_key, processed_data, data, sector, engine)
            #insert_to_fresh(file_url, data, engine)
        else:
            print(colored(f'Cant get key in {file_name}', 'yellow'))
    else:
        pass

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Validate succesfully
Insert PM_VLTH_Dong Nai_20220930 to Fresh
Insert PM_VLTH_Dong Nai_20220930 to Fresh
Insert PM_VLTH_Dong Nai_20220930 to Fresh


 25%|█████████████████████                                                               | 1/4 [00:09<00:27,  9.01s/it]

Validate succesfully
Insert PM_VLTH_HCM_20220930 to Fresh
Insert PM_VLTH_HCM_20220930 to Fresh
Insert PM_VLTH_HCM_20220930 to Fresh


 50%|██████████████████████████████████████████                                          | 2/4 [00:16<00:16,  8.17s/it]

Validate succesfully
Insert PM_VLTH_Long An_20220930 to Fresh
Insert PM_VLTH_Long An_20220930 to Fresh
Insert PM_VLTH_Long An_20220930 to Fresh


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:24<00:07,  7.86s/it]

Validate succesfully
Insert PM_VLTH_Binh Duong_20220930 to Fresh
Insert PM_VLTH_Binh Duong_20220930 to Fresh
Insert PM_VLTH_Binh Duong_20220930 to Fresh


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:30<00:00,  7.65s/it]


Đang vướng khúc hàm tracking_db, chỗ lấy sector có vấn đề

### SEND EMAIL

In [11]:
list_df_flat = [df_flat_apt, df_flat_vlth]
#Email notify missing in dictionary
if len(df_dict) != 0:
    print(colored('Missing values in dictionary','yellow'))
    df_noti_html = convert_df_to_html(type_html = 2, df = df_dict, cnxn = engine)
    to_email = ['nngocphuonguyen@savills.com.vn']
    run_email(email_type = 2, user_email = to_email, df_noti_html = df_noti_html)
else:
    pass
#Email notify imported data
df_flat_html, df_query_html = convert_df_to_html(type_html = 3, list_df = list_df_flat, type_sector = 2, cnxn = engine)
to_email = ['nngocphuonguyen@savills.com.vn']
run_email(type_sector = 'PRODUCT MIX', email_type = 3, user_email = to_email, df_flat_html = df_flat_html, df_query_html = df_query_html)

KeyError: 'Sector'

### TRACKING AUDIT

In [ ]:
#insert_to_tracking(list_df_flat, 'Tracking_Product_Mix', engine)